# Importacion de librerias y conexión

In [1]:
import mysql.connector
import pandas as pd
import numpy as np
import regex as re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
conn = mysql.connector.connect(
    host='212.227.90.6',
    user='Equipo19',
    password='E1q2u3i4p5o19',
    database='Equip_19'
)

cursor = conn.cursor()

cursor.execute("SHOW TABLES")
tablas = [t[0] for t in cursor.fetchall()]


dfs = {tabla: pd.read_sql(f"SELECT * FROM {tabla}", conn) for tabla in tablas}

df = dfs['Tourist_Accommodation10112025']

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   apartment_id                 10000 non-null  int64  
 1   name                         9997 non-null   object 
 2   description                  9862 non-null   object 
 3   host_id                      10000 non-null  int64  
 4   neighbourhood_name           10000 non-null  object 
 5   neighbourhood_district       6079 non-null   object 
 6   room_type                    10000 non-null  object 
 7   accommodates                 10000 non-null  int64  
 8   bathrooms                    9926 non-null   object 
 9   bedrooms                     9930 non-null   object 
 10  beds                         9955 non-null   float64
 11  amenities_list               9983 non-null   object 
 12  price                        9746 non-null   float64
 13  minimum_nights   

# Tratamiento de nulos: Precio

In [4]:
df.isnull().sum()

apartment_id                      0
name                              3
description                     138
host_id                           0
neighbourhood_name                0
neighbourhood_district         3921
room_type                         0
accommodates                      0
bathrooms                        74
bedrooms                         70
beds                             45
amenities_list                   17
price                           254
minimum_nights                    0
maximum_nights                    0
has_availability                550
availability_30                   0
availability_60                   0
availability_90                   0
availability_365                  0
number_of_reviews                 0
first_review_date              2604
last_review_date               2605
review_scores_rating           2709
review_scores_accuracy         2718
review_scores_cleanliness      2712
review_scores_checkin          2723
review_scores_communication 

In [5]:
precio_invalido = df['price'].isnull() | (df['price'] == 0)

df_precios_invalidos = df.loc[precio_invalido].copy()

# Comprobar los precios_inválidos
print(f"Total de registros con precio inválido (Nulos O Cero): {len(df_precios_invalidos)}")
print(f"Filas con precio nulo (NaN): {df_precios_invalidos['price'].isnull().sum()}")
print(f"Filas con precio a cero (0): {(df_precios_invalidos['price'] == 0).sum()}")


Total de registros con precio inválido (Nulos O Cero): 254
Filas con precio nulo (NaN): 254
Filas con precio a cero (0): 0


# Conversión de Tipos

CONVERSIÓN DE FECHAS

In [6]:
df["first_review_date"] = pd.to_datetime(df["first_review_date"], errors="coerce")
df["last_review_date"] = pd.to_datetime(df["last_review_date"], errors="coerce")
df["insert_date"] = pd.to_datetime(df["insert_date"], errors="coerce")

CONVERSIÓN A BOOLEANO

In [7]:
df['has_availability'] = df['has_availability'].replace({'VERDADERO': True})
df['has_availability'] = df['has_availability'].astype(bool)

df['is_instant_bookable'] = df['is_instant_bookable'].replace({'VERDADERO': True, 'FALSO': False})
df['is_instant_bookable'] = df['is_instant_bookable'].astype(bool)

CONVERSIÓN A INTEGRO

In [8]:
df['bathrooms'] = pd.to_numeric(df['bathrooms'], errors='coerce').astype('Int64')
df["bedrooms"] = pd.to_numeric(df["bedrooms"], errors='coerce').astype('Int64')
df['beds'] = pd.to_numeric(df['beds'], errors='coerce').astype('Int64')
df['is_instant_bookable'] = df['is_instant_bookable'].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   apartment_id                 10000 non-null  int64         
 1   name                         9997 non-null   object        
 2   description                  9862 non-null   object        
 3   host_id                      10000 non-null  int64         
 4   neighbourhood_name           10000 non-null  object        
 5   neighbourhood_district       6079 non-null   object        
 6   room_type                    10000 non-null  object        
 7   accommodates                 10000 non-null  int64         
 8   bathrooms                    9926 non-null   Int64         
 9   bedrooms                     9930 non-null   Int64         
 10  beds                         9955 non-null   Int64         
 11  amenities_list               9983 non-null

# Duplicados

COMPROBACIÓN DUPLICADOS APARTMENT_ID

In [9]:
df['apartment_id'].duplicated().any()

np.True_

In [10]:
total_duplicados = df['apartment_id'].duplicated().sum()
print(f"Total de registros duplicados encontrados en 'apartment_id': {total_duplicados}")

Total de registros duplicados encontrados en 'apartment_id': 350


ELIMINACIÓN DE DUPLICADOS DE APARTMENT_ID, MANTENIENDO LOS MÁS RECIENTES SEGÚN INSERT_DATE

In [11]:
df[df['apartment_id'].duplicated(keep=False)]

df = df.sort_values(['apartment_id', 'insert_date'])

df = df.drop_duplicates(subset='apartment_id', keep='last')

In [12]:
df['apartment_id'].duplicated().any()

np.False_

In [13]:
df.shape

(9650, 35)

# Estandarización de categorías y texto

NEIGHBOURHOOD

In [14]:
def clean_neighbourhood(text):
    if pd.isna(text) or text == '':
        return pd.NA
    text = str(text)
    #text = text.replace("�", "")
    text = re.sub(r"[^\p{L}\p{N}\s'\-]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = text.title()
    
    return text if text else pd.NA

In [15]:
failed_text_name = []
for i in df['neighbourhood_name']:
    if i.__contains__("�"):
        failed_text_name.append(i)

In [16]:
name_mapping = {
    'C�rmenes': 'Cármenes',
    'Torroella de Fluvi�': 'Torroella de Fluvià',
    "el Camp d'en Grassot i Gr�cia Nova": "el Camp d'en Grassot i Gràcia Nova",
    'Alc�dia': 'Alcúdia',
    "Castell� d'Emp�ries": "Castelló d'Empúries",
    'Ciudad Jard�n': 'Ciudad Jardín',
    'la Vila de Gr�cia': 'la Vila de Gràcia',
    'Ni�o Jes�s': 'Niño Jesús',
    'la Sagrada Fam�lia': 'la Sagrada Família',
    'el Putxet i el Farr�': 'el Putxet i el Farró',
    'el Barri G�tic': 'el Barri Gòtic',
    'S�ller': 'Sóller',
    'Santany�': 'Santanyí',
    'la Vila Ol�mpica del Poblenou': 'la Vila Olímpica del Poblenou',
    'San Ferm�n': 'San Fermín',
    'Llub�': 'Llubí',
    'San Juli�n': 'San Julián',
    'San Bartolom�': 'San Bartolomé',
    'Calvi�': 'Calvià',
    'Pollen�a': 'Pollença',
    'Encarnaci�n, Regina': 'Encarnación, Regina',
    'Cadaqu�s': 'Cadaqués',
    'Mah�n': 'Mahón',
    'Sarri�': 'Sarrià',
    'Palam�s': 'Palamós',
    'Celr�': 'Celrà',
    'Torroella de Montgr�': 'Torroella de Montgrí',
    'Can Bar�': 'Can Baró',
    'Arg�elles': 'Argüelles',
    'Llan��': 'Llançà',
    'Pac�fico': 'Pacífico',
    'Proven�als del Poblenou': 'Provençals del Poblenou',
    'Marratx�': 'Marratxí',
    'Sant Feliu de Gu�xols': 'Sant Feliu de Guíxols',
    'Diagonal Mar i el Front Mar�tim del Poblenou': 'Diagonal Mar i el Front Marítim del Poblenou',
    'Santa Mar�a del Cam�': 'Santa María del Camí',
    'Pont�s': 'Pontós',
    'Le�n XIII, Los Naranjos': 'León XIII, Los Naranjos',
    'Sant Lloren� des Cardassar': 'Sant Llorenç des Cardassar',
    'Sant Llu�s': 'Sant Lluís',
    'Sant Gen�s dels Agudells': 'Sant Genís dels Agudells',
    'Art�': 'Artà',
    'el Baix Guinard�': 'el Baix Guinardó',
    'Tortell�': 'Tortellà',
    'Alar�': 'Alaró',
    'Hispanoam�rica': 'Hispanoamérica',
    'Cornell� del Terri': 'Cornellà del Terri',
    'B�ger': 'Búger',
    'Llad�': 'Lladó',
    'Palau de Santa Eul�lia': 'Palau de Santa Eulàlia',
    'el Congr�s i els Indians': 'el Congrés i els Indians',
    'el Bes�s i el Maresme': 'el Besòs i el Maresme',
    'Dey�': 'Deià',
    'Casco Hist�rico de Vallecas': 'Casco Histórico de Vallecas',
    'Carretera de Carmona, Mar�a Auxiliadora, Fontanal': 'Carretera de Carmona, María Auxiliadora, Fontanal',
    'Ur�s': 'Urús',
    'Sant Mart� Vell': 'Sant Martí Vell',
    'Vila�r': 'Vilafran',
    'Vidr�': 'Vidré',
    'Sant Mart� de Proven�als': 'Sant Martí de Provençals',
    'el Guinard�': 'el Guinardó',
    'Concepci�n': 'Concepción',
    'Bescan�': 'Bescanó',
    'Jer�nimos': 'Jerónimos',
    'Moscard�': 'Moscardó',
    'Doctor Barraquer, G. Renfe, Policl�nico': 'Doctor Barraquer, G. Renfe, Policlínico',
    'Seriny�': 'Serinyà',
    "Cru�lles, Monells i Sant Sadurn� de l'Heura": "Cruïlles, Monells i Sant Sadurní de l'Heura",
    'Montu�ri': 'Montuïri',
    'el Tur� de la Peira': 'el Turó de la Peira',
    'Ventall�': 'Ventalló',
    'Fontarr�n': 'Fontarrón',
    'San Andr�s': 'San Andrés',
    'El Roc�o': 'El Rocío',
    "Bellcaire d'Empord�": "Bellcaire d'Empordà",
    'Nervi�n': 'Nervión',
    'Santa Eug�nia': 'Santa Eugènia',
    'Besal�': 'Besalú',
    'Nueva Espa�a': 'Nueva España',
    'Ull�': 'Ullà',
    'Pe�agrande': 'Peñagrande',
    'Casco Hist�rico de Barajas': 'Casco Histórico de Barajas',
    'San Jos� Obrero': 'San José Obrero',
    'La Palmilla, Doctor Mara��n': 'La Palmilla, Doctor Marañón',
    'Barrio Le�n': 'Barrio León',
    'Puigcerd�': 'Puigcerdà',
    'El Tard�n, El Carmen': 'El Tardón, El Carmen',
    'Hell�n': 'Hellín',
    'Opa�el': 'Opañel',
    'Sant Mart� de Ll�mena': 'Sant Martí de Llémena',
    'Foix�': 'Foixà',
    'Casco Hist�rico de Vic�lvaro': 'Casco Histórico de Vicálvaro',
    "La Tallada d'Empord�": "La Tallada d'Empordà",
    'Vallfogona de Ripoll�s': 'Vallfogona de Ripollès',
    'Campdev�nol': 'Campdevànol',
    'Rab�s': 'Rabós',
    'Regenc�s': 'Regencós',
    'Zof�o': 'Zofío',
    'Sant Pau de Seg�ries': 'Sant Pau de Segúries',
    'Torre Bar�': 'Torre Baró',
    'Tiro de L�nea, Santa Genoveva': 'Tiro de Línea, Santa Genoveva',
    'Tim�n': 'Timón',
    'Ma�anet de la Selva': 'Maçanet de la Selva',
    'Entrev�as': 'Entrevías',
    'Parlav�': 'Parlavà',
    'Sant Juli� de Ramis': 'Sant Julià de Ramis',
    'Mai� de Montcal': 'Maçanet de Montcal',
    'B�scara': 'Bàscara',
    'Arb�cies': 'Arbúcies',
    'Vilaju�ga': 'Vilajuïga'
}

In [17]:
df["neighbourhood_name"] = df["neighbourhood_name"].replace(name_mapping)

In [18]:
df['neighbourhood_district'] = df['neighbourhood_district'].fillna("")
df['neighbourhood_district']

0                    
1              Latina
2       Casco Antiguo
3            Eixample
4                    
            ...      
9995                 
9996                 
9997                 
9998         Chamber�
9999           Centro
Name: neighbourhood_district, Length: 9650, dtype: object

In [19]:
failed_text_district=[]
for i in df['neighbourhood_district']:
    if i.__contains__("�"):
        failed_text_district.append(i)

In [20]:
name_mapping_district = {
    'Sant Mart�': 'Sant Martí',
    'Gr�cia': 'Gràcia',
    'Sarri�-Sant Gervasi': 'Sarrià-Sant Gervasi',
    'Sants-Montju�c': 'Sants-Montjuïc',
    'Horta-Guinard�': 'Horta-Guinardó',
    'Chamart�n': 'Chamartín',
    'Tetu�n': 'Tetuán',
    'Nervi�n': 'Nervión',
    'Chamber�': 'Chamberí',
    'Vic�lvaro': 'Vicálvaro'
}

In [21]:
df["neighbourhood_district"] = df["neighbourhood_district"].replace(name_mapping_district)

In [22]:
df['neighbourhood_name'] = df['neighbourhood_name'].apply(clean_neighbourhood)
df['neighbourhood_district'] = df['neighbourhood_district'].apply(clean_neighbourhood)

In [23]:
df[['neighbourhood_name', 'neighbourhood_district']].head(10)

,neighbourhood_name,neighbourhood_district
0,Centro,<NA>
1,Cármenes,Latina
2,San Vicente,Casco Antiguo
3,L'Antiga Esquerra De L'Eixample,Eixample
4,Quart,<NA>
5,Torroella De Fluvià,<NA>
6,El Camp De L'Arpa Del Clot,Sant Martí
7,La Dreta De L'Eixample,Eixample
8,Embajadores,Centro
9,El Camp D'En Grassot I Gràcia Nova,Gràcia


In [24]:
df["neighbourhood_name"]

0                                Centro
1                              Cármenes
2                           San Vicente
3       L'Antiga Esquerra De L'Eixample
4                                 Quart
                     ...               
9995               Santa Cristina D'Aro
9996                        Palafrugell
9997                            Palamós
9998                          Trafalgar
9999                        Universidad
Name: neighbourhood_name, Length: 9650, dtype: object

In [25]:
df["neighbourhood_district"]

0                <NA>
1              Latina
2       Casco Antiguo
3            Eixample
4                <NA>
            ...      
9995             <NA>
9996             <NA>
9997             <NA>
9998         Chamberí
9999           Centro
Name: neighbourhood_district, Length: 9650, dtype: object

ESTANDARIZAR CITY

In [26]:
df['city'].value_counts()

city
barcelona    2719
madrid       2134
mallorca     1583
girona       1485
valencia      513
malaga        504
sevilla       494
menorca       218
Name: count, dtype: int64

In [27]:
city_mapping = {
    'Seville': 'Sevilla',
    'Madrid': 'Madrid',
    'Barcelona': 'Barcelona',
    'Valencia': 'Valencia',
    'Malaga': 'Málaga',
    'Málaga': 'Málaga',        
    'València': 'Valencia',    
    'Sevilla': 'Sevilla',
    'barcelona': 'Barcelona',
    'MADRID': 'Madrid',
    'valencia': 'Valencia',
    'malaga': 'Málaga',
    'sevilla': 'Sevilla',
    'girona': 'Girona',
    'Girona': 'Girona',
    'mallorca': 'Mallorca',
    'Mallorca': 'Mallorca',
    'Menorca': 'Menorca',
    'menorca': 'Menorca',
    'madrid': 'Madrid',
    'barcelona': 'Barcelona',
    'valencia': 'Valencia',
    'malaga': 'Málaga',
    'sevilla': 'Sevilla',
    'girona': 'Girona',
    'mallorca': 'Mallorca',
    'menorca': 'Menorca',
}

In [28]:
valid_cities = {
    'Barcelona',
    'Madrid',
    'Sevilla',
    'Valencia',
    'Málaga',
    'Girona',
    'Mallorca',
    'Menorca'
}

In [29]:
df['city_clean'] = df['city'].astype(str).str.strip()

# Crea una versión en minúsculas del mapeo para la coincidencia que no distinga entre mayúsculas y minúsculas.
mapping_lower = {k.lower(): v for k, v in city_mapping.items()}

# Normalizar
df['city_normalized'] = df['city_clean'].str.lower().map(mapping_lower)

# Filtrar solo cidades válidas
df = df[df['city_normalized'].isin(valid_cities)]

# Substituir colmuna original
df['city'] = df['city_normalized']
df = df.drop(columns=['city_clean', 'city_normalized'])

In [30]:
df[['city']].head(5)

,city
0,Málaga
1,Madrid
2,Sevilla
3,Barcelona
4,Girona


ESTANDARIZAR COUNTRY

In [31]:
df['country'] = 'Spain'
df[['country']].head(5)

,country
0,Spain
1,Spain
2,Spain
3,Spain
4,Spain


# Tratamiento de valores nulos

PRICE, CITY Y ROOM_TYPE

In [32]:
# Marcar inválidos
df['is_valid'] = True
df.loc[df['price'].isna() | df['city'].isna() | df['room_type'].isna(), 'is_valid'] = False
df_valid = df[df['is_valid']].copy()

In [33]:
df = df_valid

In [34]:
df.shape

(9409, 36)

Nulos en variables reviews


In [35]:
# Forzar nulos en reviews si no hay reviews
review_cols = [
    'first_review_date', 'last_review_date',
    'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin',
    'review_scores_communication', 'review_scores_location',
    'review_scores_value', 'reviews_per_month'
]
mask_no_reviews = df_valid['number_of_reviews'] == 0
df_valid.loc[mask_no_reviews, review_cols] = np.nan

# Imputar reviews_per_month
def calculate_reviews_per_month(row):
    if row['number_of_reviews'] > 0:
        if not pd.isna(row['first_review_date']):
            today = datetime.now()
            days_since_first = (today - row['first_review_date']).days
            if days_since_first > 0:
                return row['number_of_reviews'] / (days_since_first / 30)
        elif not pd.isna(row['last_review_date']):
            today = datetime.now()
            days_since_last = (today - row['last_review_date']).days
            if days_since_last > 0:
                return row['number_of_reviews'] / (days_since_last / 30)
    return np.nan

df_valid['reviews_per_month'] = df_valid.apply(calculate_reviews_per_month, axis=1)

# Amenities_list
df_valid['amenities_list'] = df_valid['amenities_list'].apply(lambda x: [] if pd.isna(x) or (isinstance(x, str) and x.strip() == '') else x)

# Validar coherencia
review_cols_no_nulls = df_valid.loc[(df_valid['number_of_reviews'] == 0) & (df_valid[review_cols].notna().any(axis=1))]
print(f"Registros inconsistentes: {len(review_cols_no_nulls)}")

Registros inconsistentes: 0


# Coherencia entre variables

COMPROBAR QUE MIN NIGHTS < MAX NIGHTS

In [36]:
df_error = df[df['minimum_nights'] > df['maximum_nights']]
df_error
# Sin resultados, así que todo está correcto

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid


COMPROBAR AVAILABILITY

In [37]:
df_hav = df[
    (df['availability_30'] > df['availability_60']) &
    (df['availability_60'] > df['availability_90']) &
    (df['availability_90'] > df['availability_365'])
].copy()

df_hav

# Sin resultados, así que todo está correcto

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid


In [38]:
cols_disp = ['availability_30', 'availability_60', 'availability_90', 'availability_365']

condicion_invalida = (
    (df['has_availability'] == True) & 
    (df[cols_disp].isna().all(axis=1))
)

df_invalidas = df[condicion_invalida]
print(df_invalidas)

# Empty Dataframe, así que todo está correcto

Empty DataFrame
Columns: [apartment_id, name, description, host_id, neighbourhood_name, neighbourhood_district, room_type, accommodates, bathrooms, bedrooms, beds, amenities_list, price, minimum_nights, maximum_nights, has_availability, availability_30, availability_60, availability_90, availability_365, number_of_reviews, first_review_date, last_review_date, review_scores_rating, review_scores_accuracy, review_scores_cleanliness, review_scores_checkin, review_scores_communication, review_scores_location, review_scores_value, is_instant_bookable, reviews_per_month, country, city, insert_date, is_valid]
Index: []

[0 rows x 36 columns]


LIMPIEZA AVAILABILITY

In [39]:
# Comprobamos coherencia entre los datos de las columnas has_availability y las cols_disp:
df_disponibilidad = df.loc[df['has_availability'] != True, 
                     ['has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365']]

df_disponibilidad
# No hay coherencia en estos casos

,has_availability,availability_30,availability_60,availability_90,availability_365
7,False,28,55,85,360
12,False,7,23,53,328
29,False,13,43,72,347
32,False,5,11,37,234
36,False,8,14,25,224
...,...,...,...,...,...
4762,False,16,46,76,76
4771,False,6,15,31,114
4791,False,13,43,73,163
4802,False,3,33,63,338


In [40]:
# Se define la condición de disponibilidad:
sidisponible = (df[cols_disp] > 0).any(axis=1)

# Se corrige la columna 'has_availability' en función de las cols_disp:
df.loc[(df['has_availability'] == False) & sidisponible, 'has_availability'] = True

In [41]:
# Comprobamos:
df[['has_availability'] + cols_disp].head()

,has_availability,availability_30,availability_60,availability_90,availability_365
0,True,7,20,40,130
1,True,0,0,0,162
2,True,26,31,31,270
3,True,9,23,49,300
4,True,0,19,49,312


COHERENCIA FECHAS INSERT_DATE ANTES QUE FIRST_REVIEW_DATE

In [42]:
df_fechas = df[
    (df['insert_date'] > df['first_review_date']) 
].copy()

totaldf = len(df_fechas)
print("Hay", totaldf, "casos en los que la 'insert_date' es posterior a la 'first_review_date', \n por lo que deducimos que se trata de anuncios que han sido actualizados y conservan reseñas antiguas.")


Hay 2804 casos en los que la 'insert_date' es posterior a la 'first_review_date', 
 por lo que deducimos que se trata de anuncios que han sido actualizados y conservan reseñas antiguas.


COHERENCIA FECHAS FIRST_REVIEW DATE ANTES QUE LAST_REVIEW_DATE

In [43]:
df_fechas2 = df[
    (df['first_review_date'] > df['last_review_date'])
].copy()

totaldf2 = len(df_fechas2)

print("Hay", totaldf2, "casos en los que la 'first_review_date' es posterior " \
"a la 'last_review_date', \n por lo que puede que haya habido un error en la adquisición de los datos") 

Hay 107 casos en los que la 'first_review_date' es posterior a la 'last_review_date', 
 por lo que puede que haya habido un error en la adquisición de los datos


In [44]:
# Se intercambian los datos entre las columnas first_review_date' y 'last_review_date' en los casos de incoherencia para deshacer esta inconsistencia  
def arreglar_fechas_reviews(row):
    if row['first_review_date'] > row['last_review_date']:
        row['first_review_date'], row['last_review_date'] = row['last_review_date'], row['first_review_date']
    return row

df = df.apply(arreglar_fechas_reviews, axis=1)

# Cambio de la presentación del dato

CAMBIO DE SCORES RATING A 0 - 100 Y TODOS LOS DEMÁS A 0 - 10

In [45]:
df['review_scores_rating'] = df['review_scores_rating'] / 10
df['review_scores_accuracy'] = df['review_scores_accuracy'] / 10
df['review_scores_checkin'] = df['review_scores_checkin'] / 10
df['review_scores_cleanliness'] = df['review_scores_cleanliness'] / 10
df['review_scores_communication'] = df['review_scores_communication'] / 10
df['review_scores_location'] = df['review_scores_location'] / 10
df['review_scores_value'] = df['review_scores_value'] / 10


In [46]:
df['review_scores_value'].head(10)

0    10.0
1     9.0
2    10.0
3     9.0
4    10.0
5    10.0
6     9.0
7     9.0
8     9.0
9     9.0
Name: review_scores_value, dtype: float64

# Tratamiento de más valores Nulos

ELIMINACIÓN NULOS Y 0 EN BEDROOMS

In [47]:
bedrooms_nulas = df.loc[df['bedrooms'].isnull(), ['bedrooms', 'room_type', 'accommodates']]
print("Hay", len(bedrooms_nulas), "casos nulos en 'bedrooms'.")


Hay 70 casos nulos en 'bedrooms'.


In [48]:
bedrooms_cero = df.loc[df['bedrooms'] == 0, ['bedrooms', 'room_type', 'accommodates']]
print("Hay", len(bedrooms_cero), "casos con 'bedrooms' igual a 0.")

Hay 296 casos con 'bedrooms' igual a 0.


In [49]:
# 1. Private room, Shared room, Hotel room → imputar como 1 si bedrooms es nulo o 0
mask_private = df['room_type'].isin(['Private room', 'Shared room', 'Hotel room']) & (
    (df['bedrooms'].isnull()) | (df['bedrooms'] == 0)
)
df.loc[mask_private, 'bedrooms'] = 1

# 2. Entire home/apt → imputar como ceil(accommodates/2) si bedrooms es nulo o 0
mask_entire = (df['room_type'] == 'Entire home/apt') & (
    (df['bedrooms'].isnull()) | (df['bedrooms'] == 0)
)
df.loc[mask_entire, 'bedrooms'] = np.ceil(df.loc[mask_entire, 'accommodates'] / 2)

# Verificación rápida
print("Casos con 'bedrooms' igual a 0:", (df['bedrooms'] == 0).sum())
print("Casos nulos restantes en 'bedrooms':", df['bedrooms'].isnull().sum())

Casos con 'bedrooms' igual a 0: 0
Casos nulos restantes en 'bedrooms': 0


ELIMINACIÓN NULOS y 0 EN BATHROOMS

In [50]:
bathrooms_cero = df.loc[df['bathrooms'] == 0]
print("Hay", len(bathrooms_cero), "casos de baños 0.")

Hay 21 casos de baños 0.


In [51]:
bathrooms_nulas = df.loc[df['bathrooms'].isnull()]
print("Hay", len(bathrooms_nulas), "casos nulos en 'bathrooms'.")

Hay 71 casos nulos en 'bathrooms'.


In [52]:
moda_bathrooms = df['bathrooms'].mode()[0]

# Imputar nulos con la moda
df['bathrooms'] = df['bathrooms'].fillna(moda_bathrooms)

# Imputar también los casos con 0 baños con la moda
df.loc[df['bathrooms'] == 0, 'bathrooms'] = moda_bathrooms

print("Casos con 'bathrooms' igual a 0:", (df['bathrooms'] == 0).sum())
print("Casos nulos restantes en 'bathrooms':", df['bathrooms'].isnull().sum())

Casos con 'bathrooms' igual a 0: 0
Casos nulos restantes en 'bathrooms': 0


ELIMINACIÓN NULOS EN BEDS (se usa la mediana del grupo según las variables room_type, bedrooms y accomodates)

In [53]:
beds_nulos = df.loc[df['beds'].isnull()]
print("Hay", len(beds_nulos), "casos nulos en 'beds'.")

Hay 43 casos nulos en 'beds'.


In [54]:
beds_ceros = df.loc[df['beds'] == 0]
print("Hay", len(beds_ceros), "casos de 'beds' 0.")

Hay 113 casos de 'beds' 0.


In [55]:
def rellenar_nulos_y_ceros_con_mediana_grupo(serie):
    """
    Calcula la mediana de la serie y la usa para rellenar
    los valores NaN y los iguales a 0.
    """
    mediana = serie.median()
    return serie.fillna(mediana).replace(0, mediana)

# Imputación con tu lógica de mediana por grupo
df['camas_imputadas'] = df.groupby(['room_type', 'bedrooms', 'accommodates'])['beds'] \
                          .transform(lambda s: s.fillna(s.median()).replace(0, s.median()))

# Sustitución
df['beds'] = df['camas_imputadas']
df = df.drop(columns=['camas_imputadas'])

# Asegurar mínimo de 1 cama
df.loc[df['beds'] == 0, 'beds'] = 1

In [56]:
print("Casos con 'beds' igual a 0:", (df['beds'] == 0).sum())
print("Casos nulos restantes en 'beds':", df['beds'].isnull().sum())

Casos con 'beds' igual a 0: 0
Casos nulos restantes en 'beds': 0


ELIMINACIÓN DE NULOS EN REVIEWS

In [57]:
colsrev = [
    'reviews_per_month',
    'number_of_reviews',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_checkin',
    'review_scores_cleanliness',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value'
]


df_filtrado = df[df['number_of_reviews'] > 0]

medianas_por_ciudad = df_filtrado.groupby('city')[colsrev].median().round(1)

medianas_por_ciudad

,reviews_per_month,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_value
city,,,,,,,,,
Barcelona,0.2,15.0,93.0,10.0,10.0,9.0,10.0,10.0,9.0
Girona,0.1,5.0,94.0,10.0,10.0,10.0,10.0,10.0,9.0
Madrid,0.2,16.0,95.0,10.0,10.0,10.0,10.0,10.0,9.0
Mallorca,0.1,6.0,96.0,10.0,10.0,10.0,10.0,10.0,9.0
Menorca,0.1,7.0,95.0,10.0,10.0,9.0,10.0,10.0,9.0
Málaga,0.2,15.0,94.0,10.0,10.0,10.0,10.0,10.0,9.0
Sevilla,0.4,32.0,95.0,10.0,10.0,10.0,10.0,10.0,9.0
Valencia,0.3,19.0,94.0,10.0,10.0,10.0,10.0,10.0,9.0


In [58]:
colsrev = [
    'reviews_per_month',
    'number_of_reviews',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_checkin',
    'review_scores_cleanliness',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value'
]

# 1. Filtramos solo alojamientos con reseñas > 0 para calcular la mediana
df_filtrado = df[df['number_of_reviews'] > 0]

# 2. Calculamos la mediana por ciudad
medianas_por_ciudad = df_filtrado.groupby('city')[colsrev].median()

# 3. Imputamos directamente en df
for col in colsrev:
    if col == 'number_of_reviews':
        # Rellenar nulos con la mediana de la ciudad y redondear hacia arriba
        df[col] = df.groupby('city')[col].transform(
            lambda s: s.fillna(medianas_por_ciudad.loc[s.name, col]).apply(np.ceil)
        )
    else:
        # Rellenar nulos con la mediana de la ciudad (sin redondeo)
        df[col] = df.groupby('city')[col].transform(
            lambda s: s.fillna(medianas_por_ciudad.loc[s.name, col])
        )

# Verificación rápida
print(df[colsrev].isnull().sum())

reviews_per_month              0
number_of_reviews              0
review_scores_rating           0
review_scores_accuracy         0
review_scores_checkin          0
review_scores_cleanliness      0
review_scores_communication    0
review_scores_location         0
review_scores_value            0
dtype: int64


In [59]:
notas = [
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_checkin',
    'review_scores_cleanliness',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value'
]

# Calcular mínimos y máximos de cada columna
min_max_global = df[notas].agg(['min', 'max']).T

print(min_max_global)

                              min    max
review_scores_rating         20.0  100.0
review_scores_accuracy        2.0   10.0
review_scores_checkin         2.0   10.0
review_scores_cleanliness     2.0   10.0
review_scores_communication   2.0   10.0
review_scores_location        2.0   10.0
review_scores_value           2.0   10.0


In [60]:
df

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid
0,11964,A ROOM WITH A VIEW,Private bedroom in our attic apartment. Right ...,45553,Centro,<NA>,Private room,2,2,1,...,10.0,10.0,10.0,10.0,0,0.405827,Spain,Málaga,2018-07-31,True
1,21853,Bright and airy room,We have a quiet and sunny room with a good vie...,83531,Cármenes,Latina,Private room,1,1,1,...,10.0,10.0,8.0,9.0,0,0.244203,Spain,Madrid,2020-01-10,True
2,32347,Explore Cultural Sights from a Family-Friendly...,Open French doors and step onto a plant-filled...,139939,San Vicente,Casco Antiguo,Entire home/apt,4,1,2,...,10.0,10.0,10.0,10.0,1,0.835843,Spain,Sevilla,2019-07-29,True
3,35379,Double 02 CasanovaRooms Barcelona,Room at a my apartment. Kitchen and 2 bathroom...,152232,L'Antiga Esquerra De L'Eixample,Eixample,Private room,2,2,1,...,10.0,10.0,10.0,9.0,1,4.264849,Spain,Barcelona,2020-01-10,True
4,35801,Can Torras Farmhouse Studio Suite,Lay in bed & watch sunlight change the mood of...,153805,Quart,<NA>,Private room,5,1,2,...,10.0,10.0,10.0,10.0,0,0.207135,Spain,Girona,2019-02-19,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,47710189,Beautiful house in the centre in Santa Cristin...,Beautiful house in the center of Santa Cristin...,263841355,Santa Cristina D'Aro,<NA>,Entire home/apt,8,2,4,...,10.0,10.0,10.0,9.0,1,0.069930,Spain,Girona,2021-01-31,True
9996,47752964,Costa Brava - Palafrugell - playa y monta�a,Piso con acceso directo a piscina. Lugar muy t...,282214688,Palafrugell,<NA>,Entire home/apt,7,2,3,...,10.0,10.0,10.0,9.0,0,0.069930,Spain,Girona,2021-02-27,True
9997,47792016,MODERN AND BRIGHT NEW FLAT IN THE CENTER OF PA...,Modern and new apartment in the old town of Pa...,263841355,Palamós,<NA>,Entire home/apt,6,1,3,...,10.0,10.0,10.0,9.0,1,0.069930,Spain,Girona,2021-01-31,True
9998,47884481,MIT House Olavide III in Madrid,The apartment is in a characteristic building ...,377605855,Trafalgar,Chamberí,Entire home/apt,4,1,1,...,10.0,10.0,10.0,9.0,0,0.181319,Spain,Madrid,2021-02-09,True


FIRST_REVIEW_DATE Y LAST_REVIEW_DATE:

Como tenemos varias columnas con uno u otro pero con varias reviews, no podemos poner la misma fecha de first review que de last review y viceversa, por lo que dejamos nulo el valor que ya lo es.

In [61]:
# Debería devolver 0 si está todo coherente
invalid_cases = df[
    (df['first_review_date'].isnull() & df['last_review_date'].notnull()) |
    (df['first_review_date'].notnull() & df['last_review_date'].isnull())
]
len(invalid_cases)


3028

In [62]:
df.loc[
    df['last_review_date'].isnull() & df['first_review_date'].notnull(),
    ['first_review_date', 'last_review_date', 'number_of_reviews', 'reviews_per_month']
]

,first_review_date,last_review_date,number_of_reviews,reviews_per_month
1,2014-10-10,NaT,33,0.244203
2,2011-05-01,NaT,148,0.835843
5,2011-04-06,NaT,25,0.140528
9,2011-04-06,NaT,120,0.674536
12,2012-02-11,NaT,9,0.053721
...,...,...,...,...
9811,2020-10-08,NaT,2,0.032189
9832,2020-02-07,NaT,2,0.028463
9833,2020-01-08,NaT,4,0.056127
9877,2020-12-07,NaT,3,0.049889


In [63]:
mask = df['last_review_date'].isnull() & df['first_review_date'].notnull()
df.loc[mask & (df['number_of_reviews'] == 1), 'last_review_date'] = df.loc[mask & (df['number_of_reviews'] == 1), 'first_review_date']


In [64]:
df.loc[
    df['first_review_date'].isnull() & df['last_review_date'].notnull(),
    ['first_review_date', 'last_review_date', 'number_of_reviews', 'reviews_per_month']
]

,first_review_date,last_review_date,number_of_reviews,reviews_per_month
3,NaT,2020-04-01,292,4.264849
7,NaT,2017-05-06,33,0.317817
11,NaT,2016-01-10,189,1.576314
19,NaT,2017-09-12,154,1.547220
32,NaT,2017-02-04,416,3.892701
...,...,...,...,...
9879,NaT,2020-09-08,2,0.031679
9880,NaT,2020-06-11,4,0.060514
9889,NaT,2020-01-10,3,0.042135
9900,NaT,2020-11-10,3,0.049153


In [65]:
mask = df['last_review_date'].isnull() & df['first_review_date'].notnull()
df.loc[mask & (df['number_of_reviews'] == 1), 'last_review_date'] = df.loc[mask & (df['number_of_reviews'] == 1), 'first_review_date']


COMPROBACIÓN NULOS DE NEIGHBOURHOOD DISTRICT

In [66]:
df.loc[df["neighbourhood_district"].isna(), "neighbourhood_name"].value_counts()

df_nulls = df[df["neighbourhood_district"].isna()][["neighbourhood_name", "city", "apartment_id"]]

df_nulls

,neighbourhood_name,city,apartment_id
0,Centro,Málaga,11964
4,Quart,Girona,35801
5,Torroella De Fluvià,Girona,48764
27,Palma De Mallorca,Mallorca,164621
30,Alcúdia,Mallorca,178456
...,...,...,...
9992,Blanes,Girona,47101215
9993,Centro,Málaga,47357284
9995,Santa Cristina D'Aro,Girona,47710189
9996,Palafrugell,Girona,47752964


In [67]:
df[["neighbourhood_district", "neighbourhood_name"]]

,neighbourhood_district,neighbourhood_name
0,<NA>,Centro
1,Latina,Cármenes
2,Casco Antiguo,San Vicente
3,Eixample,L'Antiga Esquerra De L'Eixample
4,<NA>,Quart
...,...,...
9995,<NA>,Santa Cristina D'Aro
9996,<NA>,Palafrugell
9997,<NA>,Palamós
9998,Chamberí,Trafalgar


# Nuevas columnas

CREACIÓN DE NUEVA COLUMNA

In [68]:
df['ocupacion_mes'] = 30 - df['availability_30']

# EDA

COMPROBACIÓN OUTLIERS

In [69]:
pd.set_option('display.max_columns', None)
df[df['price'] > 6000]

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,beds,amenities_list,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review_date,last_review_date,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid,ocupacion_mes
2390,12970898,APARTAMENTO CENTRICO-EL CARMEN,Apartamento totalmente reformado en 2018. Dos ...,71363084,La Xerea,Ciutat Vella,Entire home/apt,6,2,3,7.0,"Paid parking on premises, Shampoo, Oven, Host ...",6071.0,2,60,True,0,0,3,3,38,2018-09-03,NaT,93.0,10.0,9.0,10.0,10.0,10.0,9.0,0,0.433460,Spain,Valencia,2020-09-28,True,30
8996,37153661,Unique Luxury Apartment 6People Pool & Facilities,250 square meters - Exclusive luxury apartment...,42407188,Mirasierra,Fuencarral - El Pardo,Entire home/apt,6,5,5,5.0,"Iron, Coffee maker, Fire extinguisher, Wifi, T...",25857.0,3,1125,True,18,47,77,279,3,2019-05-08,NaT,90.0,10.0,10.0,10.0,10.0,10.0,9.0,0,0.037768,Spain,Madrid,2020-09-13,True,12
9269,39350327,2 habitaciones y Ba�o privado en Barrio Salamanca,Brand new floor (daylight basement) with 2 roo...,13669384,Fuente Del Berro,Salamanca,Private room,4,1,2,2.0,"High chair, Iron, Garden or backyard, Private ...",7143.0,2,1125,True,0,0,0,23,8,NaT,2020-01-03,95.0,9.0,10.0,10.0,10.0,10.0,10.0,1,0.111993,Spain,Madrid,2020-09-13,True,30
9537,41372147,"VILLA VERD, nice villa in the country.","Villa Verd is a beautiful country house, finis...",313785862,Alcúdia,<NA>,Entire home/apt,6,2,3,5.0,"Iron, TV, Kitchen, Hair dryer, Free street par...",28571.0,1,1125,True,30,60,63,194,0,NaT,NaT,96.0,10.0,10.0,10.0,10.0,10.0,9.0,1,0.079740,Spain,Mallorca,2020-08-28,True,0
9647,42188834,Roman Room by Hotel Mam�,<b>The space</b><br />Looks like a small cinem...,333351874,Palma De Mallorca,<NA>,Private room,2,1,1,1.0,"Hair dryer, Shampoo, Air conditioning, Bathtub...",25286.0,1,1125,True,0,0,0,267,0,NaT,NaT,96.0,10.0,10.0,10.0,10.0,10.0,9.0,1,0.079740,Spain,Mallorca,2020-08-28,True,30


# Eliminacion de outliers de precio superior a 20.000

In [70]:
df = df.drop(df[df['price'] > 20000].index)

In [71]:
pd.set_option('display.max_columns', None)
df[df['beds'] > 25]

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,beds,amenities_list,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review_date,last_review_date,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid,ocupacion_mes
9435,40550295,Idyllischer Fincaurlaub f�r Biker inkl. Fr�hst�ck,Das Agroturismo Finca Son Miranda befindet sic...,11125218,Sencelles,<NA>,Private room,16,13,12,28.0,"First aid kit, Indoor fireplace, Wifi, Free pa...",1710.0,3,1125,True,29,59,89,364,0,NaT,NaT,96.0,10.0,10.0,10.0,10.0,10.0,9.0,0,0.07974,Spain,Mallorca,2020-10-21,True,1


# Capping variables para eliminar outliers en bedrooms, beds y bathrooms

In [72]:
df = df.copy()
# 1. Definir las variables a procesar
variables_a_limitar = ['bathrooms', 'bedrooms', 'beds']

print("--- 📏 Aplicando Capping SOLO al Límite Superior por IQR (1.5) ---")

for columna in variables_a_limitar:
    print(f"\nProcesando columna: '{columna}'")
    
    # 2. Calcular Q1, Q3, e IQR
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    
    # 3. Definir SOLO el Límite Superior
    limite_superior = Q3 + 1.5 * IQR
    
    # --- 4. Aplicar el Capping Superior ---
    
    # Identificar registros que superan el límite superior
    registros_sup = df[columna] > limite_superior
    total_limitados = registros_sup.sum()
    
    # Acotar valores superiores
    df[columna] = np.where(
        df[columna] > limite_superior,
        limite_superior, # Si es mayor, sustituirlo por el límite superior
        df[columna]
    )
    
    print(f"  Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
    print(f"  Límite Superior (Capping): {limite_superior:.2f}")
    print(f"  **Total de {columna} limitados: {total_limitados} (solo superiores)**")

print("\n--- ✅ Proceso de Capping completado, evitando la limitación inferior innecesaria. ---")

--- 📏 Aplicando Capping SOLO al Límite Superior por IQR (1.5) ---

Procesando columna: 'bathrooms'
  Q1: 1.00, Q3: 2.00, IQR: 1.00
  Límite Superior (Capping): 3.50
  **Total de bathrooms limitados: 357 (solo superiores)**

Procesando columna: 'bedrooms'
  Q1: 1.00, Q3: 3.00, IQR: 2.00
  Límite Superior (Capping): 6.00
  **Total de bedrooms limitados: 25 (solo superiores)**

Procesando columna: 'beds'
  Q1: 1.00, Q3: 4.00, IQR: 3.00
  Límite Superior (Capping): 8.50
  **Total de beds limitados: 161 (solo superiores)**

--- ✅ Proceso de Capping completado, evitando la limitación inferior innecesaria. ---


# Exportacion de csv

In [73]:
df.to_csv('tourist_accommodation_10112025_clean.csv', index=False)

In [74]:
df.isnull().sum()

apartment_id                      0
name                              3
description                     127
host_id                           0
neighbourhood_name                0
neighbourhood_district         3654
room_type                         0
accommodates                      0
bathrooms                         0
bedrooms                          0
beds                              0
amenities_list                    0
price                             0
minimum_nights                    0
maximum_nights                    0
has_availability                  0
availability_30                   0
availability_60                   0
availability_90                   0
availability_365                  0
number_of_reviews                 0
first_review_date              6440
last_review_date               6503
review_scores_rating              0
review_scores_accuracy            0
review_scores_cleanliness         0
review_scores_checkin             0
review_scores_communication 